# Object Oriented Programming in R

## Environments as objects

Inspired by RC, R6 and _proto_ classes (objects) we present how we can define our own object/class system. 

We start by defining objects. For that we use standard R environment. 

In [3]:
objectCreator = function(vars=list()) {
    envir = new.env()
    for (vn in names(vars)) {
        assign(vn, vars[[vn]], envir = envir)
        ## Alternative way
        ## env[[vn]] <- vars[[vn]]
    }
    envir
}

complexNum <- objectCreator(
    vars=list(
        x=1, 
        y=2
    )
)

print(complexNum$x)
print(complexNum$y)

print(ls.str(complexNum))


[1] 1
[1] 2
x :  num 1
y :  num 2


### Adding Methods

What we have done so far could be easyly achived by using list. But if we wnat to have methods, is where environment comes handy. Lets try.

In [4]:
cn <- objectCreator(
    vars=list(
        x=1, 
        y=2,
        re=function() {
            # This won't work
            x
        }
    )
)

print(cn$re())

ERROR: Error in cn$re(): object 'x' not found


The problem is the environment of this function. 

In [5]:
print(environment(cn$re))

<environment: R_GlobalEnv>


So let assigin to environment of the function the environment of the object.

In [6]:
objectCreator = function(vars=list()) {
    envir = new.env()
    for (vn in names(vars)) {
        envir[[vn]] <- vars[[vn]]
        if (is.function(envir[[vn]])) {
          environment(envir[[vn]]) <- envir
        }
    }
    envir
}

cn <- objectCreator(
    vars=list(
        x=1, 
        y=2,
        re=function() {
            x
        }
    )
)

print(cn$re())


[1] 1


This was a getter. Now time of setting an x. Let's project out complex number to x asis.

In [7]:
cn <- objectCreator(
    vars=list(
        x=1, 
        y=2,
        projectX=function() {
            # This won't work
            y <- 0
        }
    )
)

cn$projectX()
print(cn$y)

[1] 2


In [8]:
cn <- objectCreator(
    vars=list(
        x=1, 
        y=2,
        projectX=function() {
            # This won't work
            y <- 0
            environment()
        }
    )
)

env <- cn$projectX()
print(env)
print(parent.env(env))
print(cn)

<environment: 0x2dcc910>
<environment: 0x2cc8138>
<environment: 0x2cc8138>


In [9]:
cn <- objectCreator(
    vars=list(
        x=1, 
        y=2,
        projectX=function() {
            y <<- 0
        }
    )
)
cn$projectX()
print(cn$y)

[1] 0


## Clasess

Now it is time for classes. Class is a set of objects that share the same attributes and method. Something like:
Class of complex number is a set of _things_ that have an attribute _x_, _y_, and method _re_, _im_ etc......

In order to implement it we need two functions. First a _function_ that transform a recipe into a function that later can produce for us objects that agree with this recipe (instances of this class).

In [10]:
classGenerator <- function(ClassList) {
    function() objectCreator(vars=ClassList)
}

And that is it. Let's see it in action.

In [11]:
ComplexNumber <- list(
    x=0,
    y=0
)
   
cn1 <- classGenerator(ComplexNumber)()
cn2 <- classGenerator(ComplexNumber)()
print(cn1$x)
print(cn2$x)

[1] 0
[1] 0


### RefClass

That was a bit silly. So let's how we construct objects using RefClass that generate objects that have a attribue being a data.table (we suppose that it contains a column dt (for datatame and metric)). Also method dt that return first column and metric, that returns the second.

In [12]:
library('data.table')

TimeSeries <- setRefClass(
    "XXX",
    fields=list(
        dataTable="data.table"
    ),
    methods=list(
        initialize=function(dataTable=data.table()) {
            dataTable <<- dataTable
        },
        dt=function() dataTable[, dt],
        metric=function() dataTable[, metric]
    )
)
            

timeSeries <- TimeSeries$new(data.table(dt=c("2015-10-01", "2015-10-02"), metric=c(10, 20)))
timeSeries$dt()
timeSeries$metric()
class(timeSeries)

[1] "2015-10-01" "2015-10-02"

[1] 10 20

[1] "XXX"
attr(,"package")
[1] ".GlobalEnv"

So let implement this. We change the name of setRefClass into setRClass.

In [13]:
setRClass <- function(
    className, #not used at the moment,
    fields=list(),
    methods=list()
) {
    objectVars <- c(fields, methods)
    classVars <- list(
        objectVars=objectVars,
        new=function(...) {
            newObject <- objectCreator(objectVars)
            
            initVars = list(...)
            
            if ("initialize" %in% names(objectVars)) {
                newObject$initialize(...)
                newObject$initialize <- NULL
            }
            newObject
        }
    )
    objectCreator(classVars) 
}

TimeSeries <- setRClass(
    "XXX",
    fields=list(
        dataTable="data.table"
    ),
    methods=list(
        initialize=function(dataTable=data.table()) {
            print(dataTable)
            dataTable <<- dataTable
        },
        dt=function() dataTable[, dt],
        metric=function() dataTable[, metric]
    )
)
    
timeSeries <- TimeSeries$new(data.table(dt=c("2015-10-01", "2015-10-02"), metric=c(10, 20)))
timeSeries$dataTable
timeSeries$dt()
timeSeries$metric()
class(timeSeries)

           dt metric
1: 2015-10-01     10
2: 2015-10-02     20


,dt,metric
1,2015-10-01,10
2,2015-10-02,20


[1] "2015-10-01" "2015-10-02"

[1] 10 20

[1] "environment"

There are few differences ...

## Metaprogramming

### Adding methods to existing class

In [14]:
EmptyClass <- R6::R6Class()
EmptyClass$new()$newMethod()

ERROR: Error in eval(expr, envir, enclos): attempt to apply non-function


In [15]:
EmptyClass$private_fields
EmptyClass$set("public", "newMethod", function() 1000)
EmptyClass$new()$newMethod()

NULL

[1] 1000

### Call method dynamically

In [16]:
library('R6')
library('data.table')

DataTable <- R6Class(
    lock_objects=FALSE,
    public=list(
        dataTable=NULL,
        initialize=function(dataTable) {
            self$dataTable <- dataTable
        },
        ids=function() self$dataTable[, id],
        dates=function() self$dataTable[, date]
    )
)

dt1 <- DataTable$new(data.table(id=1:10, date=seq(as.Date("2015-10-01"), as.Date("2015-10-10"), "day")))
dt1$ids()
method_to_call <- "ids"
dt1[[method_to_call]]()


[1]  1  2  3  4  5  6  7  8  9 10

[1]  1  2  3  4  5  6  7  8  9 10

### Modifying $ and introducing method_missing

In [17]:
t <- new.env()
t$a <- 1
t$a

c <- function() {
  old <- `$`
  '$' <- function(t, a) {
    print(old(t, a))
    123
  }
  t$a <- 2
  t$a
}

c()

t$a


[1] 1

[1] 2


[1] 123

[1] 2

## S3 Class